In [ ]:
# Lets import libraries which will be required during the process
import pandas as pd
import numpy as np
import cv2
import pydicom
import matplotlib.pyplot as plt
import skimage.io as io

from skimage import exposure
from IPython.display import display
from datetime import date

from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager

from pathlib import Path
import json
import os
import shutil
import random
from tqdm import tqdm
import png
import re

today = date.today()

Lets define a root path.

In [ ]:
root_path = Path('/home/adil/Desktop/PhD_Data/PhD_Courses/First_Semester/Machine_Learning/Breast_Cancer_Detection/CBIS_DDSM/')

In [ ]:
file = pd.read_csv(str(root_path / 'CBIS_DDSM_Metadata.csv'))
print(file.columns) 
file.head()

Index(['patient_id', 'split', 'left_or_right breast', 'image_view',
       'roi_subject_id', 'roi_file_size', 'roi_file_location',
       'raw_subject_id', 'raw_file_size', 'raw_file_location',
       'abnormality_id', 'abnormality_type', 'calc_type', 'calc_distribution',
       'mass_shape', 'mass_margins', 'assessment', 'breast_density',
       'pathology', 'roi_file_location_new', 'raw_file_location_new'],
      dtype='object')


,patient_id,split,left_or_right breast,image_view,roi_subject_id,roi_file_size,roi_file_location,raw_subject_id,raw_file_size,raw_file_location,...,abnormality_type,calc_type,calc_distribution,mass_shape,mass_margins,assessment,breast_density,pathology,roi_file_location_new,raw_file_location_new
0,P_00038,Test,LEFT,CC,Calc-Test_P_00038_LEFT_CC_1,14.06 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC_1/08-29-...,Calc-Test_P_00038_LEFT_CC,27.84 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-20...,...,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,NaN,NaN,4,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC_1/08-29-...,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-20...
1,P_00038,Test,LEFT,MLO,Calc-Test_P_00038_LEFT_MLO_1,14.62 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1/08-29...,Calc-Test_P_00038_LEFT_MLO,28.97 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2...,...,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,NaN,NaN,4,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1/08-29...,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2...
2,P_00038,Test,RIGHT,CC,Calc-Test_P_00038_RIGHT_CC_1,13.41 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29...,Calc-Test_P_00038_RIGHT_CC,25.73 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,...,calcification,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN_WITHOUT_CALLBACK,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...
3,P_00038,Test,RIGHT,CC,Calc-Test_P_00038_RIGHT_CC_2,13.24 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_2/08-29...,Calc-Test_P_00038_RIGHT_CC,25.73 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,...,calcification,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN_WITHOUT_CALLBACK,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_2/08-29...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...
4,P_00038,Test,RIGHT,MLO,Calc-Test_P_00038_RIGHT_MLO_1,15.17 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1/08-2...,Calc-Test_P_00038_RIGHT_MLO,27.64 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO/08-29-...,...,calcification,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN_WITHOUT_CALLBACK,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1/08-2...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO/08-29-...


CSV contains three classes: BENIGN, MALIGNANT and BENIGN_WITHOUT_CALLBACK. We need two classes. Lets convert BENIGN_WITHOUT_CALLBACK into BENIGN Class

In [ ]:
file.loc[file['pathology']=='BENIGN_WITHOUT_CALLBACK', 'pathology'] = 'BENIGN'
file.head()

,patient_id,split,left_or_right breast,image_view,roi_subject_id,roi_file_size,roi_file_location,raw_subject_id,raw_file_size,raw_file_location,...,abnormality_type,calc_type,calc_distribution,mass_shape,mass_margins,assessment,breast_density,pathology,roi_file_location_new,raw_file_location_new
0,P_00038,Test,LEFT,CC,Calc-Test_P_00038_LEFT_CC_1,14.06 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC_1/08-29-...,Calc-Test_P_00038_LEFT_CC,27.84 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-20...,...,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,NaN,NaN,4,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC_1/08-29-...,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-20...
1,P_00038,Test,LEFT,MLO,Calc-Test_P_00038_LEFT_MLO_1,14.62 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1/08-29...,Calc-Test_P_00038_LEFT_MLO,28.97 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2...,...,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,NaN,NaN,4,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1/08-29...,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2...
2,P_00038,Test,RIGHT,CC,Calc-Test_P_00038_RIGHT_CC_1,13.41 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29...,Calc-Test_P_00038_RIGHT_CC,25.73 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,...,calcification,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...
3,P_00038,Test,RIGHT,CC,Calc-Test_P_00038_RIGHT_CC_2,13.24 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_2/08-29...,Calc-Test_P_00038_RIGHT_CC,25.73 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,...,calcification,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_2/08-29...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...
4,P_00038,Test,RIGHT,MLO,Calc-Test_P_00038_RIGHT_MLO_1,15.17 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1/08-2...,Calc-Test_P_00038_RIGHT_MLO,27.64 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO/08-29-...,...,calcification,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1/08-2...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO/08-29-...


Lets define the final Targt class which will be the combination of abnormality type and pathology

In [ ]:
file['final_target'] = file['abnormality_type'].str.upper() + '_' + file['pathology']
file

,patient_id,split,left_or_right breast,image_view,roi_subject_id,roi_file_size,roi_file_location,raw_subject_id,raw_file_size,raw_file_location,...,calc_type,calc_distribution,mass_shape,mass_margins,assessment,breast_density,pathology,roi_file_location_new,raw_file_location_new,final_target
0,P_00038,Test,LEFT,CC,Calc-Test_P_00038_LEFT_CC_1,14.06 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC_1/08-29-...,Calc-Test_P_00038_LEFT_CC,27.84 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-20...,...,PUNCTATE-PLEOMORPHIC,CLUSTERED,NaN,NaN,4,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC_1/08-29-...,./CBIS-DDSM/Calc-Test_P_00038_LEFT_CC/08-29-20...,CALCIFICATION_BENIGN
1,P_00038,Test,LEFT,MLO,Calc-Test_P_00038_LEFT_MLO_1,14.62 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1/08-29...,Calc-Test_P_00038_LEFT_MLO,28.97 MB,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2...,...,PUNCTATE-PLEOMORPHIC,CLUSTERED,NaN,NaN,4,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO_1/08-29...,./CBIS-DDSM/Calc-Test_P_00038_LEFT_MLO/08-29-2...,CALCIFICATION_BENIGN
2,P_00038,Test,RIGHT,CC,Calc-Test_P_00038_RIGHT_CC_1,13.41 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29...,Calc-Test_P_00038_RIGHT_CC,25.73 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,...,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,CALCIFICATION_BENIGN
3,P_00038,Test,RIGHT,CC,Calc-Test_P_00038_RIGHT_CC_2,13.24 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_2/08-29...,Calc-Test_P_00038_RIGHT_CC,25.73 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,...,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_2/08-29...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2...,CALCIFICATION_BENIGN
4,P_00038,Test,RIGHT,MLO,Calc-Test_P_00038_RIGHT_MLO_1,15.17 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1/08-2...,Calc-Test_P_00038_RIGHT_MLO,27.64 MB,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO/08-29-...,...,VASCULAR,NaN,NaN,NaN,2,2.0,BENIGN,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO_1/08-2...,./CBIS-DDSM/Calc-Test_P_00038_RIGHT_MLO/08-29-...,CALCIFICATION_BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,P_02092,Train,LEFT,CC,Mass-Training_P_02092_LEFT_CC_1,11.78 MB,./CBIS-DDSM/Mass-Training_P_02092_LEFT_CC_1/07...,Mass-Training_P_02092_LEFT_CC,31.11 MB,./CBIS-DDSM/Mass-Training_P_02092_LEFT_CC/07-2...,...,NaN,NaN,IRREGULAR,SPICULATED,3,2.0,MALIGNANT,./CBIS-DDSM/Mass-Training_P_02092_LEFT_CC_1/07...,./CBIS-DDSM/Mass-Training_P_02092_LEFT_CC/07-2...,MASS_MALIGNANT
3561,P_02092,Train,LEFT,MLO,Mass-Training_P_02092_LEFT_MLO_1,14.14 MB,./CBIS-DDSM/Mass-Training_P_02092_LEFT_MLO_1/0...,Mass-Training_P_02092_LEFT_MLO,37.35 MB,./CBIS-DDSM/Mass-Training_P_02092_LEFT_MLO/07-...,...,NaN,NaN,IRREGULAR,SPICULATED,3,2.0,MALIGNANT,./CBIS-DDSM/Mass-Training_P_02092_LEFT_MLO_1/0...,./CBIS-DDSM/Mass-Training_P_02092_LEFT_MLO/07-...,MASS_MALIGNANT
3562,P_01413,Train,RIGHT,MLO,Mass-Training_P_01413_RIGHT_MLO_1,24.40 MB,./CBIS-DDSM/Mass-Training_P_01413_RIGHT_MLO_1/...,Mass-Training_P_01413_RIGHT_MLO,47.37 MB,./CBIS-DDSM/Mass-Training_P_01413_RIGHT_MLO/07...,...,NaN,NaN,IRREGULAR,ILL_DEFINED,5,2.0,MALIGNANT,./CBIS-DDSM/Mass-Training_P_01413_RIGHT_MLO_1/...,./CBIS-DDSM/Mass-Training_P_01413_RIGHT_MLO/07...,MASS_MALIGNANT
3563,P_01630,Train,RIGHT,MLO,Mass-Training_P_01630_RIGHT_MLO_1,27.02 MB,./CBIS-DDSM/Mass-Training_P_01630_RIGHT_MLO_1/...,Mass-Training_P_01630_RIGHT_MLO,53.57 MB,./CBIS-DDSM/Mass-Training_P_01630_RIGHT_MLO/07...,...,NaN,NaN,LOBULATED,ILL_DEFINED,4,2.0,BENIGN,./CBIS-DDSM/Mass-Training_P_01630_RIGHT_MLO_1/...,./CBIS-DDSM/Mass-Training_P_01630_RIGHT_MLO/07...,MASS_BENIGN


Here, if we want, we can specify the abnormality - like if we only want to work with mass, or calcification


In [ ]:
file = file[file.abnormality_type=='mass'].reset_index(drop=True)
file.abnormality_type.unique()

array(['mass'], dtype=object)

Lets work with both of these. We can see that there are two columns about file location, one is simple file_location and other is 
file_location_new. The later one is the complete path to the dicom files. We are going to use that one. Moreover, if we check the 
length of raw_file_location_new and its unique values, we will see they are not the same. This is because, there are images which contain more than one abnormality.


### **Train/Validaton/Test Split**

Lets split the DataFrame into Train/Validation/Test Split. We will define a function which takes a row and modified the split as the Training/Validation/Testing. First we will separate the validation files. We will just select the first 300 test files when we will be using both classes and 200 when we will be working on one class.

In [ ]:
validation_set = file[file.split=='Test']['raw_file_location_new'].unique()[:200].tolist()
len(validation_set), validation_set[:2]

(200,
 ['./CBIS-DDSM/Mass-Test_P_00017_LEFT_CC/10-04-2016-DDSM-NA-12683/1.000000-full mammogram images-82967/1-1.dcm',
  './CBIS-DDSM/Mass-Test_P_00017_LEFT_MLO/10-04-2016-DDSM-NA-89998/1.000000-full mammogram images-29934/1-1.dcm'])

In [ ]:
def test_val_split(row, validation_files_names):
    """
    
    The function takes a row of a dataframe, and check if the row['raw_file_location_new']
    is in validation_file_names. If yes, it will modify the Split.
    
    """
    if row['raw_file_location_new'] in validation_files_names:
        return 'Validation'
    else:
        return row['split']

In [ ]:
file['split'] = file.apply(lambda row: test_val_split(row, validation_set), axis=1)
file['split'].unique()

array(['Validation', 'Test', 'Train'], dtype=object)

Lets count the number of instances in each split. 

In [ ]:
file.groupby(['split']).size()

split
Test           167
Train         1318
Validation     209
dtype: int64

## **Pre-processing**

In [ ]:
len(file.raw_file_location_new), len(file.raw_file_location_new.unique())

(1694, 1590)

Lets get the unique files. 

In [ ]:
unique_files = file.raw_file_location_new.unique()
unique_files[:4]

array(['./CBIS-DDSM/Mass-Test_P_00017_LEFT_CC/10-04-2016-DDSM-NA-12683/1.000000-full mammogram images-82967/1-1.dcm',
       './CBIS-DDSM/Mass-Test_P_00017_LEFT_MLO/10-04-2016-DDSM-NA-89998/1.000000-full mammogram images-29934/1-1.dcm',
       './CBIS-DDSM/Mass-Test_P_00032_RIGHT_CC/10-04-2016-DDSM-NA-31069/1.000000-full mammogram images-83371/1-1.dcm',
       './CBIS-DDSM/Mass-Test_P_00032_RIGHT_MLO/10-04-2016-DDSM-NA-21963/1.000000-full mammogram images-19361/1-1.dcm'],
      dtype=object)

Lets define a path where we will store the CoCo files

In [ ]:
coco_path = str(root_path / "coco_files_mass/")
os.makedirs(coco_path, exist_ok=True)

# lets shuffle the unique files before splitting into train and test
random.shuffle(unique_files)
train_files = file[file.split=='Train'].raw_file_location_new.unique().tolist()
train_split = ['_'.join(x.split('/')[1:])[:-3] for x in train_files]
val_files   = file[file.split=='Validation'].raw_file_location_new.unique().tolist()
val_split   = ['_'.join(x.split('/')[1:])[:-3] for x in val_files]
test_files  = file[file.split=='Test'].raw_file_location_new.unique().tolist()
test_split  = ['_'.join(x.split('/')[1:])[:-3] for x in test_files]
len(train_split), len(val_split), len(test_split), len(train_split)+len(val_split)+len(test_split), len(unique_files)

(1231, 200, 159, 1590, 1590)

### **CoCO Dictionary**
Lets define a coco dictionary function to use later. 

In [ ]:
def coco_dictionary():
    """
    
    A funtion to create the CoCo Dictionary
    CoCo_Dict: 'info'         : {},
                'images'      : {},
                'licenses'    : {},
                'annotations' : {}.
                'categories'  : {}
    
    """
    coco_dict = {
    # Lets file the info
    'info' : {
        
            'description':"COCO 2014 Like Dataset",
            'url':""   ,
            'version':"1.0"                  ,
            'year':2014                      ,
            'contributor':"Muhammad Adil"  ,
            'date_created':"04/04/2023"      , 

              },
        'images'      : [],
        'licenses'    : [],
        'annotations' : [],
        'categories'  : [],
                }
    classes = ['MASS_BENIGN', 'MASS_MALIGNANT'] #, 'CALCIFICATION_BENIGN', 'CALCIFICATION_MALIGNANT']
    for idx, clas in enumerate(classes):
        coco_dict['categories'].append(
            {'Supercategory': 'Breast_Cancer', 'id': idx, 'name': clas}
            )
    return coco_dict


Lets use the above function to initialize the train and validation dictionaries. We will also initialize the index values for annotatios and images. 

In [ ]:
train_coco    = coco_dictionary()
val_coco      = coco_dictionary()
test_coco     = coco_dictionary()
train_idx_ann = 0
val_idx_ann   = 0
test_idx_ann  = 0
train_idx_img = 0
val_idx_img   = 0
test_idx_img  = 0
train_coco['categories']

[{'Supercategory': 'Breast_Cancer', 'id': 0, 'name': 'MASS_BENIGN'},
 {'Supercategory': 'Breast_Cancer', 'id': 1, 'name': 'MASS_MALIGNANT'}]

## **Utilities**
Lets define all the Utilities which will be required during the process. These utilities include:
- Extraction of metadata
- Converting DICOM to PNG

There are few corrupted files in the dataset, we will implement try:except block to skip those corrupted filesThere are few corrupted files in the dataset, we will implement try:except block to skip those corrupted files


In [ ]:
def convert_dicom_roi_to_png(dicom_file, outfile):
    """
    
    Upon providing the Dicom Region Of Interest Path - The function 
    will read the region of interest file and write the pixel array data to 
    outfile and save it as PNG File.
    args: dicom_roi : Dicom Region Of interest file Path
                    : outfile : the file to write the region of interest arrays data
                    
    
    """
    try:
        # lets read the dicom file
        dicom_file = pydicom.dcmread(dicom_file)

        dicom_file_shape = dicom_file.pixel_array.shape

        img_2d = dicom_file.pixel_array.astype(float)

        # lets rescale the gray values between 0-255
        img_2d_scaled = np.uint8((np.maximum(img_2d, 0) / img_2d.max()) * 255)

        with open(outfile, 'wb') as png_file:
            writer = png.Writer(dicom_file_shape[1], dicom_file_shape[0], greyscale=True)
            writer.write(png_file, img_2d_scaled)
    except Exception as e:
        return False
    
    return True
        
def convert_dicom_img_to_png(dicom_file, outfile):
    """
    
    Upon providing the Dicom Image - The function 
    will read the image and write the pixel array data to 
    outfile and save it as PNG File.
    args: dicom_roi : Dicom Image file Path
                    : outfile : the file to write the region of interest arrays data
                    
    
    """
    try:
        # lets read the dicom file
        dicom_file = pydicom.dcmread(dicom_file)

        dicom_file_shape = dicom_file.pixel_array.shape

        img_2d = dicom_file.pixel_array.astype(float)

        # lets rescale the gray values between 0-255
        img_2d_scaled = np.uint8((np.maximum(img_2d, 0) / img_2d.max()) * 256)

        with open(outfile, 'wb') as png_file:
            writer = png.Writer(dicom_file_shape[1], dicom_file_shape[0], greyscale=True)
            writer.write(png_file, img_2d_scaled)
    
    except Exception as e:
        return False
    
    return True
        
def get_metadata(file, img_f):
    """
    
    Upon providng the csv file object and image name, the function will extract the metadata and store it in the dictionary
    and will return it.
    args:  file: A csv_file object
           img_f: The name of the image file which we need the metadata
    return metadata dictionary with the metadata values
    
    """    
    
    try:
        dicom_rois_loc = file[file.raw_file_location_new==img_f]['roi_file_location_new'].tolist()
        final_target = file[file.raw_file_location_new==img_f]['final_target'].tolist()

        metadata = {}
            # 'MASS_BENIGN', 
            #             'MASS_MALIGNANT', 
        class_names = [ 
                        'MASS_BENIGN', 
                        'MASS_MALIGNANT', 

                        #'CALCIFICATION_BENIGN', 
                        #'CALCIFICATION_MALIGNANT',
                      ]
        
        abnormality_coords = []
        for dicom_roi in dicom_rois_loc:

            convert_dicom_roi_to_png(dicom_roi, 'test.png')
            roi_img = cv2.imread('./test.png', 0)
            contours, _ = cv2.findContours(roi_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            # Lets select the longest contour here - There might be some points which we dont need - we will discard those
            contours = sorted(contours, key=lambda x: len(x), reverse=True)[0]

            # lets squeeze it - we will remove the first dimension here (n,1,2) --> (n,2)
            contours = contours.squeeze()

            # lets transform it into list and append it in the above list
            contours = contours.tolist()
            abnormality_coords.append(contours)
            
        MASS_BENIGN             = []
        MASS_MALIGNANT          = []
        # CALCIFICATION_BENIGN    = []
        # CALCIFICATION_MALIGNANT = []
        
        for index, target in enumerate(final_target):
            if target == 'MASS_BENIGN':
                MASS_BENIGN.append(abnormality_coords[index])
            if target == 'MASS_MALIGNANT':
                MASS_MALIGNANT.append(abnormality_coords[index])
                
            # if target == 'CALCIFICATION_BENIGN':
            #     CALCIFICATION_BENIGN.append(abnormality_coords[index])
            # if target == 'CALCIFICATION_MALIGNANT':
            #     CALCIFICATION_MALIGNANT.append(abnormality_coords[index])
                
        metadata['MASS_BENIGN']             =   MASS_BENIGN
        metadata['MASS_MALIGNANT']          =   MASS_MALIGNANT
        
#         metadata['CALCIFICATION_BENIGN']    = CALCIFICATION_BENIGN
#         metadata['CALCIFICATION_MALIGNANT'] = CALCIFICATION_MALIGNANT

        return metadata
    
    except Exception as e:
        
        return None

def image_contrast_equalization(img_f):
    """
    
    The function will take the path of the Dicom Image - Read it and return 
    the image with Enhanced Contrast Histogram Contrast Equalization
    
    """
    try:
        dicom_image = pydicom.dcmread(img_f).pixel_array
        dicom_image_enhanced_equa = exposure.equalize_adapthist(dicom_image)

        return dicom_image_enhanced_equa
    except Exception as e:
        return None



In [ ]:
for img_f in tqdm(unique_files):

    
    data_dict = file[file['raw_file_location_new']==img_f][['left_or_right breast', 'image_view', 'abnormality_type', 'calc_type', 
                                              'calc_distribution', 'mass_shape', 'mass_margins', 'pathology']].iloc[0].to_dict()
    img_path = str(root_path / img_f)
    
    # the output name - name of the file which will be stored in the images dictionary
    out_fname = '_'.join(img_f.split('/')[1:])[:-3]
    
    
    # lets put a flag if the output filename is not in the val split or train split, we will ignore it
    if not (out_fname in train_split or out_fname in val_split or out_fname in test_split) : continue
    
    # lets save the file in the respective train or validation folder
    if out_fname in train_split:
        path_to_save = 'train/'
    if out_fname  in val_split:
        path_to_save = 'validation/'
    if out_fname in test_split:
        path_to_save = 'test/'
    os.makedirs(coco_path + '/' + path_to_save, exist_ok=True)
    # lets get the metadata and img
    
    file_path = coco_path + '/' + path_to_save + out_fname+'png'

    # convert the image into PNG format
    if not convert_dicom_img_to_png(img_f, outfile=file_path): continue
    
    metadata = get_metadata(file, img_f)
    if metadata is None: continue

    img = image_contrast_equalization(img_f)
    if img is None: continue
    

    #  lets save the info in the dictionary
    if out_fname in train_split:
        for class_name, values in metadata.items():
            for ground_truth in train_coco['categories']:
                if ground_truth['name'] == class_name:
                    class_id = ground_truth['id']
            for polygons in values:
                polygons_coord = []
                for xy_val in polygons:
                    polygons_coord.append(int(xy_val[0]))
                    polygons_coord.append(int(xy_val[1]))
                train_coco['annotations'].append({
            
                    'segmentation' : [polygons_coord],
                    'area'         : cv2.contourArea(np.array(polygons)),
                    'iscrowd'      : 0,
                    'image_id'     : train_idx_img,
                    'bbox'         : list(cv2.boundingRect(np.array(polygons))),
                    'category_id'  : class_id,
                    'id'           : train_idx_ann,
                    'Data_dict'    : data_dict,
                    
                                                }),
                train_idx_ann +=1 
        train_coco['images'].append({
            'license'         : 3,
            'file_name'       : out_fname + 'png',
            'coco_url'        : '',
            'height'          : img.shape[0],
            'width'           : img.shape[1],
            'date_captured'   : str(today),
            'flickr_url'      : '',
            'id'              : train_idx_img,
            
            
                                    }),
        train_idx_img += 1
        
    if out_fname in val_split:
        for class_name, values in metadata.items():
            for ground_truth in val_coco['categories']:
                if ground_truth['name'] == class_name:
                    class_id = ground_truth['id']
            for polygons in values:
                polygons_coord = []
                for xy_val in polygons:
                    polygons_coord.append(int(xy_val[0]))
                    polygons_coord.append(int(xy_val[1]))
                val_coco['annotations'].append({
                    
                    'segmentation' : [polygons_coord],
                    'area'         : cv2.contourArea(np.array(polygons)),
                    'iscrowd'      : 0,
                    'image_id'     : val_idx_img,
                    'bbox'         : list(cv2.boundingRect(np.array(polygons))),
                    'category_id'  : class_id,
                    'id'           : val_idx_ann,
                    'Data_dict'    : data_dict,
                         
                    
                                                }),
                val_idx_ann +=1 
        val_coco['images'].append({
            'license'         : 3,
            'file_name'       : out_fname + 'png',
            'coco_url'        : '',
            'height'          : img.shape[0],
            'width'           : img.shape[1],
            'date_captured'   : str(today),
            'flickr_url'      : '',
            'id'              : val_idx_img,
            
            
                                    }),
        val_idx_img += 1
        
    if out_fname in test_split:
        for class_name, values in metadata.items():
            for ground_truth in test_coco['categories']:
                if ground_truth['name'] == class_name:
                    class_id = ground_truth['id']
            for polygons in values:
                polygons_coord = []
                for xy_val in polygons:
                    polygons_coord.append(int(xy_val[0]))
                    polygons_coord.append(int(xy_val[1]))
                test_coco['annotations'].append({
                    
                    'segmentation' : [polygons_coord],
                    'area'         : cv2.contourArea(np.array(polygons)),
                    'iscrowd'      : 0,
                    'image_id'     : test_idx_img,
                    'bbox'         : list(cv2.boundingRect(np.array(polygons))),
                    'category_id'  : class_id,
                    'id'           : test_idx_ann,
                    'Data_dict'    : data_dict,
                         
                    
                                                }),
                test_idx_ann +=1 
        test_coco['images'].append({
            'license'         : 3,
            'file_name'       : out_fname + 'png',
            'coco_url'        : '',
            'height'          : img.shape[0],
            'width'           : img.shape[1],
            'date_captured'   : str(today),
            'flickr_url'      : '',
            'id'              : test_idx_img,
            
            
                                    }),
        test_idx_img += 1
    
          

100%|█████████████████████████████████████| 1590/1590 [1:08:10<00:00,  2.57s/it]


In [ ]:
os.makedirs(coco_path + "/" + 'annotations', exist_ok=True)
with open(coco_path + "/" + 'annotations/train.json', 'w') as outfile:
    json.dump(train_coco, outfile)
    
with open(coco_path + "/" + 'annotations/validation.json', 'w') as outfile:
    json.dump(val_coco, outfile)
    
with open(coco_path + "/" + 'annotations/test.json', 'w') as outfile:
    json.dump(test_coco, outfile)

In [ ]:
train_idx_ann, train_idx_img, val_idx_img, val_idx_ann, test_idx_img, test_idx_ann

(1258, 1174, 200, 209, 159, 167)

In [ ]:
len(train_split),len(val_split),len(train_split)+len(val_split),len(unique_files)

(1231, 200, 1431, 1590)

In [ ]:
with open('./coco_files_mass/annotations/test.json') as f:
    data = json.load(f)

In [ ]:
data['categories']

[{'Supercategory': 'Breast_Cancer', 'id': 0, 'name': 'MASS_BENIGN'},
 {'Supercategory': 'Breast_Cancer', 'id': 1, 'name': 'MASS_MALIGNANT'}]

In [ ]:
data['images'][0]

{'license': 3,
 'file_name': 'CBIS-DDSM_Mass-Test_P_01183_LEFT_CC_10-04-2016-DDSM-NA-02655_1.000000-full mammogram images-65284_1-1.png',
 'coco_url': '',
 'height': 4792,
 'width': 2696,
 'date_captured': '2023-05-09',
 'flickr_url': '',
 'id': 0}

In [ ]:
data['annotations'][0]

{'segmentation': [[1348,
   2790,
   1348,
   2791,
   1347,
   2792,
   1346,
   2793,
   1346,
   2794,
   1345,
   2795,
   1344,
   2795,
   1343,
   2795,
   1342,
   2794,
   1342,
   2793,
   1341,
   2792,
   1340,
   2792,
   1339,
   2793,
   1338,
   2793,
   1337,
   2792,
   1336,
   2791,
   1335,
   2792,
   1334,
   2793,
   1333,
   2793,
   1332,
   2793,
   1332,
   2794,
   1332,
   2795,
   1331,
   2796,
   1330,
   2797,
   1329,
   2798,
   1328,
   2799,
   1327,
   2799,
   1326,
   2799,
   1325,
   2798,
   1324,
   2798,
   1325,
   2798,
   1326,
   2799,
   1326,
   2800,
   1326,
   2801,
   1325,
   2802,
   1324,
   2802,
   1323,
   2802,
   1322,
   2801,
   1321,
   2800,
   1320,
   2800,
   1319,
   2800,
   1318,
   2800,
   1317,
   2800,
   1316,
   2799,
   1315,
   2798,
   1314,
   2797,
   1314,
   2796,
   1314,
   2795,
   1313,
   2796,
   1312,
   2797,
   1311,
   2797,
   1310,
   2797,
   1309,
   2797,
   1308,
   2796,
   1307,
   